Following along with:

https://drzgan.github.io/Python_CFD/15.%20Cavity%20flow%20with%20Naiver-Stokes%20equation.html

This is step 13.1

Using a direct poisson pressure coupling method:

\begin{equation*}
\frac{\partial{u}}{\partial{t}} + u \frac{\partial{u}}{\partial{x}} + v \frac{\partial{u}}{\partial{y}} = - \frac{1}{\rho} \frac{\partial{p}}{\partial{x}} + \mu \left(\frac{\partial^2{u}}{\partial{x}^2} + \frac{\partial^2{u}}{\partial{y^2}}\right)
\end{equation*}

\begin{equation*}
\frac{\partial{v}}{\partial{t}} + u \frac{\partial{v}}{\partial{x}} + v \frac{\partial{v}}{\partial{y}} = - \frac{1}{\rho} \frac{\partial{p}}{\partial{y}} + \mu \left(\frac{\partial^2{v}}{\partial{x}^2} + \frac{\partial^2{v}}{\partial{y^2}}\right)
\end{equation*}

\begin{equation*}
\frac{\partial^2 p}{\partial x^2}+\frac{\partial^2 p}{\partial y^2} = \rho\left(\frac{\partial}{\partial t}\left(\frac{\partial u}{\partial x} +\frac{\partial v}{\partial y} \right) - \frac{\partial u}{\partial x}\frac{\partial u}{\partial x}-2\frac{\partial u}{\partial y}\frac{\partial v}{\partial x}-\frac{\partial v}{\partial y}\frac{\partial v}{\partial y} \right)
\end{equation*}

Rearranging the momentum equations:

\begin{equation*}
\frac{\partial{u}}{\partial{t}} = - u \frac{\partial{u}}{\partial{x}} - v \frac{\partial{u}}{\partial{y}} - \frac{1}{\rho} \frac{\partial{p}}{\partial{x}} + \mu \left(\frac{\partial^2{u}}{\partial{x}^2} + \frac{\partial^2{u}}{\partial{y^2}}\right)
\end{equation*}

\begin{equation*}
\frac{\partial{v}}{\partial{t}} = - u \frac{\partial{v}}{\partial{x}} - v \frac{\partial{v}}{\partial{y}} - \frac{1}{\rho} \frac{\partial{p}}{\partial{y}} + \mu \left(\frac{\partial^2{v}}{\partial{x}^2} + \frac{\partial^2{v}}{\partial{y^2}}\right)
\end{equation*}

For this exercise, we will use:
- explicit time step
- backwards difference for convection terms
- second order centered difference for diffusion terms

That is copy and pasted below from the example notebook since i'm not retyping all of that.

\begin{split}
\begin{split}
u_{i,j}^{n+1} = u_{i,j}^{n} & - u_{i,j}^{n} \frac{\Delta t}{\Delta x} \left(u_{i,j}^{n}-u_{i-1,j}^{n}\right) - v_{i,j}^{n} \frac{\Delta t}{\Delta y} \left(u_{i,j}^{n}-u_{i,j-1}^{n}\right) \\
& - \frac{\Delta t}{\rho 2\Delta x} \left(p_{i+1,j}^{n}-p_{i-1,j}^{n}\right) \\
& + \nu \left(\frac{\Delta t}{\Delta x^2} \left(u_{i+1,j}^{n}-2u_{i,j}^{n}+u_{i-1,j}^{n}\right) + \frac{\Delta t}{\Delta y^2} \left(u_{i,j+1}^{n}-2u_{i,j}^{n}+u_{i,j-1}^{n}\right)\right)
\end{split}
\end{split}

\begin{split}
\begin{split}
v_{i,j}^{n+1} = v_{i,j}^{n} & - u_{i,j}^{n} \frac{\Delta t}{\Delta x} \left(v_{i,j}^{n}-v_{i-1,j}^{n}\right) - v_{i,j}^{n} \frac{\Delta t}{\Delta y} \left(v_{i,j}^{n}-v_{i,j-1}^{n})\right) \\
& - \frac{\Delta t}{\rho 2\Delta y} \left(p_{i,j+1}^{n}-p_{i,j-1}^{n}\right) \\
& + \nu \left(\frac{\Delta t}{\Delta x^2} \left(v_{i+1,j}^{n}-2v_{i,j}^{n}+v_{i-1,j}^{n}\right) + \frac{\Delta t}{\Delta y^2} \left(v_{i,j+1}^{n}-2v_{i,j}^{n}+v_{i,j-1}^{n}\right)\right)
\end{split}
\end{split}

\begin{split}
\begin{split}
p_{i,j}^{n} = & \frac{\left(p_{i+1,j}^{n}+p_{i-1,j}^{n}\right) \Delta y^2 + \left(p_{i,j+1}^{n}+p_{i,j-1}^{n}\right) \Delta x^2}{2\left(\Delta x^2+\Delta y^2\right)} \\
& -\frac{\rho\Delta x^2\Delta y^2}{2\left(\Delta x^2+\Delta y^2\right)} \\
& \times \left[\frac{1}{\Delta t}\left(\frac{u_{i+1,j}-u_{i-1,j}}{2\Delta x}+\frac{v_{i,j+1}-v_{i,j-1}}{2\Delta y}\right)-\frac{u_{i+1,j}-u_{i-1,j}}{2\Delta x}\frac{u_{i+1,j}-u_{i-1,j}}{2\Delta x} -2\frac{u_{i,j+1}-u_{i,j-1}}{2\Delta y}\frac{v_{i+1,j}-v_{i-1,j}}{2\Delta x}-\frac{v_{i,j+1}-v_{i,j-1}}{2\Delta y}\frac{v_{i,j+1}-v_{i,j-1}}{2\Delta y}\right]
\end{split}
\end{split}

In [2]:
import numpy as np
from matplotlib import pyplot as plt, cm
from mpl_toolkits.mplot3d import Axes3D

In [3]:
nx = 42
ny = 42
nt = 500 
nit = 50
c = 1
length = 1
dx = length / (nx-1)
dy = length / (ny-1)
x = np.linspace(0,length,nx)
y = np.linspace(0,length,ny)
X, Y = np.meshgrid(x, y)

rho = 1
mu = 0.01
dt = 0.001

u = np.zeros((ny,nx))
v = np.zeros((ny,nx))
p = np.zeros((ny,nx))
b = np.zeros((ny,nx))

print("Reynold's number =", rho*c*length/mu)

Reynold's number = 100.0


In [ ]:
# make some helper functions so it doesnt look like total shit

def build_up_b(b, rho, dt, u, v, dx, dy):
    